In [1]:
!pip install Unidecode --quiet
import unidecode 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.9/235.9 kB 6.0 MB/s eta 0:00:00


In [2]:
import numpy as np 
import re
import nltk
from nltk.corpus import stopwords

# Se cargan los corpus


In [3]:
# Montamos el drive
from google.colab import drive 
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
with open('/content/drive/Shareddrives/DL + NLP/Proyecto DL + NLP/Entrega_Final/data/generative/SCHIZOPHRENIA/corpus_schizophrenia.txt', 'r') as f:
    corpus_schizophrenia = f.read()

In [5]:
# Visualizamos el contenido del corpus de esquizofrenia
print(corpus_schizophrenia[:236])

Doesdid anyone else believe that they arewere in Hell? An alternate reality where there is no Good. I believe that I died about 3 years ago from an LSD trip, and that I am now living in an alternate reality in which God cannot reach me.


## Separamos por documentos

In [6]:
def separate_documents(corpus):
    # Inicialización de variables
    documents = []
    documents = corpus.strip().split('\n\n')

    return documents

In [7]:
doc = separate_documents(corpus_schizophrenia)

In [8]:
doc[36]

'Im kind of new at this because in recent years I was able to make some what of a normal life for myself in society with medication.'

# Creación del script del dataset

In [9]:
# Se crea un script de dataset utilizando la librería de Hugging Face
!pip install datasets --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 40.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 23.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 32.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 19.1 MB/s eta 0:00:00


In [10]:
from datasets import Dataset, DatasetDict

In [11]:
# Se crea una lista de diccionarios con los campos "id" y "text" para cada documento en doc
data = [{"id": str(i), "text": doc[i]} for i in range(len(doc))]

In [12]:
# Se crea el dataset con la lista de diccionarios, hacemos el split
train_size = int(0.8 * len(data))
val_size = int(0.1 * len(data))
test_size = len(data) - train_size - val_size

train_data = data[:train_size]
val_data = data[train_size:train_size+val_size]
test_data = data[train_size+val_size:]

dataset = DatasetDict({'train': train_data, 'validation': val_data, 'test': test_data})

In [13]:
# Obtener los conjuntos de entrenamiento del objeto dataset
train_data = dataset['train']
val_data = dataset['validation']
test_data = dataset['test']

In [14]:
import os

In [15]:
# Rutas completas donde se almacenarán los archivos de texto de los conjuntos
train_file_path = os.path.join('/content/drive/Shareddrives/DL + NLP/Proyecto DL + NLP/Entrega_Final/data/generative/SCHIZOPHRENIA/split', 'train.txt')
val_file_path = os.path.join('/content/drive/Shareddrives/DL + NLP/Proyecto DL + NLP/Entrega_Final/data/generative/SCHIZOPHRENIA/split', 'validation.txt')
test_file_path = os.path.join('/content/drive/Shareddrives/DL + NLP/Proyecto DL + NLP/Entrega_Final/data/generative/SCHIZOPHRENIA/split', 'test.txt')

In [16]:
# Abrir el archivo de texto para escribir los datos de entrenamiento
with open(train_file_path, 'w', encoding='utf-8') as f:
    for example in train_data:
        # Escribir cada ejemplo de entrenamiento en una línea separada
        f.write(example['text'] + '\n')

# Abrir el archivo de texto para escribir los datos de validacion
with open(val_file_path, 'w', encoding='utf-8') as f:
    for example in val_data:
        # Escribir cada ejemplo de validacion en una línea separada
        f.write(example['text'] + '\n')

# Abrir el archivo de texto para escribir los datos de test
with open(test_file_path, 'w', encoding='utf-8') as f:
    for example in test_data:
        # Escribir cada ejemplo de test en una línea separada
        f.write(example['text'] + '\n')

#Entrenamiento del modelo

In [17]:
!pip install transformers --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 55.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 74.2 MB/s eta 0:00:00


In [18]:
!git clone https://github.com/huggingface/transformers.git
!cp transformers/examples/pytorch/language-modeling/run_clm.py /content/transformers/examples/pytorch/language-modeling/

Cloning into 'transformers'...
remote: Enumerating objects: 141840, done.
remote: Counting objects: 100% (1877/1877), done.
remote: Compressing objects: 100% (845/845), done.
remote: Total 141840 (delta 1176), reused 1407 (delta 871), pack-reused 139963
Receiving objects: 100% (141840/141840), 140.04 MiB | 16.08 MiB/s, done.
Resolving deltas: 100% (106095/106095), done.
cp: 'transformers/examples/pytorch/language-modeling/run_clm.py' and '/content/transformers/examples/pytorch/language-modeling/run_clm.py' are the same file


In [19]:
# Configuración del entrenamiento
model_checkpoint = "gpt2" # Modelo base
output_dir = "/content/drive/Shareddrives/DL + NLP/Proyecto DL + NLP/Entrega_Final/output/generative/Schizophrenia" # Carpeta donde se guardarán los modelos y los logs de entrenamiento
num_train_epochs = 3 # Número de épocas de entrenamiento
learning_rate = 5e-5 # Tasa de aprendizaje
per_device_train_batch_size = 2 # Tamaño del lote de entrenamiento por dispositivo
per_device_eval_batch_size = 2 # Tamaño del lote de evaluación por dispositivo
warmup_steps = 500 # Número de pasos de calentamiento
logging_dir = "/content/drive/Shareddrives/DL + NLP/Proyecto DL + NLP/Entrega_Final/logs/generative/Schizophrenia" # Carpeta donde se guardarán los registros de entrenamiento
logging_steps = 500 # Frecuencia de los registros de entrenamiento
save_steps = 500 # Frecuencia de guardar el modelo

In [21]:
!pip install -r /content/transformers/examples/pytorch/language-modeling/requirements.txt --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.1/219.1 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 37.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 8.0 MB/s eta 0:00:00


In [22]:
!pip install git+https://github.com/huggingface/transformers --quiet

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [23]:
# Se instala el módulo "evaluate" que no está instalado en el entorno en el que se está ejecutando el script
!pip install evaluate --quiet

In [24]:
%run /content/transformers/examples/pytorch/language-modeling/run_clm.py \
    --model_name_or_path gpt2 \
    --train_file "/content/drive/Shareddrives/DL + NLP/Proyecto DL + NLP/Entrega_Final/data/generative/SCHIZOPHRENIA/split/train.txt" \
    --validation_file "/content/drive/Shareddrives/DL + NLP/Proyecto DL + NLP/Entrega_Final/data/generative/SCHIZOPHRENIA/split/validation.txt" \
    --do_train \
    --do_eval \
    --output_dir "/content/drive/Shareddrives/DL + NLP/Proyecto DL + NLP/Entrega_Final/output/generative/Schizophrenia" \
    --num_train_epochs 3 \
    --learning_rate 5e-5 \
    --per_device_train_batch_size 2 \
    --per_device_eval_batch_size 2 \
    --warmup_steps 500 \
    --logging_dir "/content/drive/Shareddrives/DL + NLP/Proyecto DL + NLP/Entrega_Final/logs/generative/Schizophrenia" \
    --logging_steps 500 \
    --save_steps 500 \
    --fp16 \
    --overwrite_output_dir \
    --report_to tensorboard

INFO:__main__:Training/evaluation parameters TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_backend=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=False,
do_train=True,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=None,
evaluation_strategy=no,
fp16=True,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_config={'fsdp_min_num_params': 0, 'xla': False, 'xla_fsdp_grad_ckpt': False},
fsdp_min_num_params=0,
fsdp_transformer_layer_cls_to_wrap=None,
full_determinism=False,
gradient_accumulation_steps=1,
gradient_checkpointing=False,
greater_is_better=None,
group_by_length=False,
half_precision_backend=auto,
hub_model_id=None,
hub_private_repo=

INFO:datasets.download.download_manager:Downloading took 0.0 min
INFO:datasets.download.download_manager:Checksum Computation took 0.0 min


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

INFO:datasets.builder:Generating train split


Generating train split: 0 examples [00:00, ? examples/s]

...

Step,Training Loss
500,3.636000
1000,3.491400
1500,3.361800
2000,3.294400
2500,3.238700
3000,3.166200
3500,3.164900


[INFO|trainer.py:2904] 2023-05-18 13:36:18,790 >> Saving model checkpoint to /content/drive/Shareddrives/DL + NLP/Proyecto DL + NLP/Entrega_Final/output/generative/Schizophrenia/checkpoint-500
[INFO|configuration_utils.py:458] 2023-05-18 13:36:18,802 >> Configuration saved in /content/drive/Shareddrives/DL + NLP/Proyecto DL + NLP/Entrega_Final/output/generative/Schizophrenia/checkpoint-500/config.json
[INFO|configuration_utils.py:364] 2023-05-18 13:36:18,812 >> Configuration saved in /content/drive/Shareddrives/DL + NLP/Proyecto DL + NLP/Entrega_Final/output/generative/Schizophrenia/checkpoint-500/generation_config.json
[INFO|modeling_utils.py:1836] 2023-05-18 13:36:23,178 >> Model weights saved in /content/drive/Shareddrives/DL + NLP/Proyecto DL + NLP/Entrega_Final/output/generative/Schizophrenia/checkpoint-500/pytorch_model.bin
[INFO|tokenization_utils_base.py:2181] 2023-05-18 13:36:23,187 >> tokenizer config file saved in /content/drive/Shareddrives/DL + NLP/Proyecto DL + NLP/Entreg

***** train metrics *****
  epoch                    =        3.0
  total_flos               =  3553842GF
  train_loss               =     3.3293
  train_runtime            = 0:24:43.76
  train_samples            =       2434
  train_samples_per_second =      4.921
  train_steps_per_second   =      2.461


***** eval metrics *****
  epoch                   =        3.0
  eval_accuracy           =     0.3644
  eval_loss               =     3.1353
  eval_runtime            = 0:00:22.74
  eval_samples            =        314
  eval_samples_per_second =     13.804
  eval_steps_per_second   =      6.902
  perplexity              =    22.9959


[INFO|modelcard.py:451] 2023-05-18 13:58:12,947 >> Dropping the following result as it does not have all the necessary fields:
{'task': {'name': 'Causal Language Modeling', 'type': 'text-generation'}, 'metrics': [{'name': 'Accuracy', 'type': 'accuracy', 'value': 0.3643679449103735}]}


# Generamos una muestra de texto a partir de una semilla

In [25]:
from transformers import pipeline

In [26]:
from transformers import AutoModelForCausalLM, AutoTokenizer

In [27]:
!pip install xformers --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.2/108.2 MB 9.6 MB/s eta 0:00:00


In [28]:
model_path = "/content/drive/Shareddrives/DL + NLP/Proyecto DL + NLP/Entrega_Final/output/generative/Schizophrenia/checkpoint-3500"

In [29]:
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path)

[INFO|tokenization_utils_base.py:1808] 2023-05-18 13:58:32,400 >> loading file vocab.json
[INFO|tokenization_utils_base.py:1808] 2023-05-18 13:58:32,405 >> loading file merges.txt
[INFO|tokenization_utils_base.py:1808] 2023-05-18 13:58:32,407 >> loading file tokenizer.json
[INFO|tokenization_utils_base.py:1808] 2023-05-18 13:58:32,409 >> loading file added_tokens.json
[INFO|tokenization_utils_base.py:1808] 2023-05-18 13:58:32,412 >> loading file special_tokens_map.json
[INFO|tokenization_utils_base.py:1808] 2023-05-18 13:58:32,420 >> loading file tokenizer_config.json
[INFO|configuration_utils.py:667] 2023-05-18 13:58:32,564 >> loading configuration file /content/drive/Shareddrives/DL + NLP/Proyecto DL + NLP/Entrega_Final/output/generative/Schizophrenia/checkpoint-3500/config.json
[INFO|configuration_utils.py:725] 2023-05-18 13:58:32,569 >> Model config GPT2Config {
  "_name_or_path": "/content/drive/Shareddrives/DL + NLP/Proyecto DL + NLP/Entrega_Final/output/generative/Schizophrenia/

In [30]:
# Definir la configuración del generador de texto
generator = pipeline('text-generation', model=model, tokenizer=tokenizer)

In [31]:
# Definir la semilla para la generación de texto
prompt = "Hello, how are you doing?"
# Generar texto a partir de la semilla
text = generator(prompt, max_length=150, do_sample=True, temperature=0.7)
print(text[0]['generated_text'])

[INFO|configuration_utils.py:577] 2023-05-18 13:58:37,123 >> Generate config GenerationConfig {
  "_from_model_config": true,
  "bos_token_id": 50256,
  "do_sample": true,
  "eos_token_id": 50256,
  "max_length": 50,
  "transformers_version": "4.30.0.dev0"
}

[WARNING|utils.py:1282] 2023-05-18 13:58:37,125 >> Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Hello, how are you doing? I mean, Ive been working on this in the last week, and it feels like a long and arduous process, and I feel a lot of pressure to get better. Ive been so tired and exhausted, and I just need to get better, but I cant work. I feel like Im being held back. I just want to be able to get out of here. Im just trying to get my life together. I dont want to ever leave this place. I dont want to leave this place. Im just scared. I just need to get out of here. I feel this way. Ive been so tired and drained, im just trying to get my life together. I dont want to ever
